In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Carregar o conjunto de dados Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Data Augmentation: Ruído gaussiano
================
Conjunto de técnicas que evitam Overfitting (hiper-ajuste aos dados de treinamento, ou seja, aprende-se os dados treinados, não as características do problema).

O que é Ruído Gaussiano?
_______________

* Gerado a partir de uma distribuição normal, caracterizada por uma média $\mu$ e um desvio-padrão $\sigma$;
* O ruído deve ser pequeno o suficiente para não distorcer significativamente os dados, mas grandeo suficiente para criar variações úteis

Adicão de ruído gaussiano
___________________________

* Para cada ponto de dados original, crie uma nova amostra adicionando um ruído gaussiano. A nova amostra será:

$$ X_novo = X_original + \mathcal{E}$$

Onde o ruído gaussiano $\mathcal{E}$:

* $\mathcal{E} \sim \mathcal{N}(0, \sigma^2)$
* $\sigma$ deve ser escolhido com base na escala e na sensibilidade dos dados

Escolha do valor adequado para $\sigma$
__________________________________________

* Se o desvio padrão do ruído for muito pequeno, as novas amostras serão muito semelhantes aos dados originais, oferecendo pouca diversidade;
* Se for muito grande, o ruído pode introduzir uma variabilidade que não está presente nos dados reais, potencialmente corrompendo a integridade dos dados;
* Em geral, um bom ponto de partida para o desvio padrão do ruído é cerca de 1-5% da faixa (range) de cada feature.

In [4]:
class IrisClassificationModel(nn.Module):
    def __init__(self):
        super(IrisClassificationModel, self).__init__()
        self.hidden = nn.Linear(X_train.shape[1], 16)  # Camada oculta com 16 neurônios
        self.output = nn.Linear(16, 3)  # Três classes de saída

    def forward(self, x):
        x = torch.relu(self.hidden(x))  # Função de ativação ReLU na camada oculta
        x = self.output(x)  # Saída direta (softmax será aplicado na função de perda)
        return x

model = IrisClassificationModel()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_loader_augmented:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Mostrar o progresso a cada 10 épocas
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.8353
Epoch [20/100], Loss: 0.5800
Epoch [30/100], Loss: 0.4165
Epoch [40/100], Loss: 0.4703
Epoch [50/100], Loss: 0.3460
Epoch [60/100], Loss: 0.4388
Epoch [70/100], Loss: 0.3664
Epoch [80/100], Loss: 0.3571
Epoch [90/100], Loss: 0.2510
Epoch [100/100], Loss: 0.1780


In [8]:
# Converter os dados de teste para tensores do PyTorch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Avaliar o modelo no conjunto de teste
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Accuracy on the test set: 96.67%


Data Augmentation: Mixup
================================

técnica que cria novas amostras de treinamento ao combinar pares de amostras existentes, tanto em termos de features quanto de rótulos. Essa técnica é particularmente eficaz em melhorar a robustez e a generalização do modelo, especialmente em tarefas de classificação.

Fórmulas do Mixup
_____________________
A ideia central do Mixup é criar novas amostras lineares interpolando entre dois exemplos aleatórios do conjunto de dados.
* Para duas amostras $(x_i , y_i)$ e $(x_j , y_j)$, uma nova amostra $(\tilde{x}, \tilde{y})$ é criada da seguinte forma:

$$\tilde{x} = \lambda x_i + (1 - \lambda) x_j$$
$$\tilde{y} = \lambda y_i + (1 - \lambda) y_j$$

Onde $\lambda$ é um valor retirado de uma distribuição beta $\beta (\alpha , \alpha)$, com $\alpha$ sendo um hiperparâmetro que contra a mistura

Data Augmentation: SMOTE (Synthetic Minority Over-sampling Technique)
====================================================================

Técnica de aumento de dados que gera novos exemplos sintéticos para a classe minoritária, com o objetivo de equilibrar o conjunto de dados.